# Introduction to Feature Selection

-----

Feature selection helps you in the mission to create an accurate predictive model. It helps you by choosing features that will give you as good or better accuracy while requiring less data.

The key benefits of performing feature selection on the data are:

- Reduces Overfitting: Less redundant data means less opportunity to make decisions based on noise.
- Improves Accuracy: Less misleading data means modeling accuracy improves.
- Reduces Training Time: Less data means that algorithms train faster.
- Improves Interpretability: Less complexity of a model and makes it easier to interpret.

In some cases, a domain expert can indicate which features have the most predictive power and which features can be ignored. When this is not possible (and in some cases even when it is possible), we can employ algorithmic feature selection to automatically quantify the importance of features so that a threshold can be used to identify the best features for a particular application.

Broadly speaking there are three general classes of feature selection algorithms:
- filter methods
- wrapper methods
- embedded methods.

The scikit learn provides a number of [feature selection algorithms][skfs] that implement these techniques. The rest of this notebook explores them in more detail.

-----

[wfs]: https://en.wikipedia.org/wiki/Feature_selection
[skfs]: http://scikit-learn.org/stable/modules/feature_selection.html

## Table of Contents

[Data](#Data)

[Filter Methods](#Filter_Methods)
- [Statistical Tests](#Statistical-Tests)

- [Univariate Techniques](#Univariate-Techniques)

[Wrapper methods](#Wrapper-methods)
- [Recursive Feature Extraction](#Recursive-Feature-Extraction)

[Embedded Methods](#Embedded-Methods)
- [Select From Model](#Select-From-Model)

-----

Before proceeding with the rest of this notebook, we first have our standard notebook setup code.

-----

In [1]:
# Set up Notebook

%matplotlib inline

# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

# Set global fiugure properties
import matplotlib as mpl
mpl.rcParams.update({'axes.titlesize' : 20,
                     'axes.labelsize' : 18,
                     'legend.fontsize': 16})

# Set default seaborn plotting style
sns.set_style('white')

# Some cells take a while to run, so we will time them
from time import time

-----
[[Back to TOC]](#Table-of-Contents)

## Data

To perform feature selection, we need representative data. In this section we introduce the two data sets that we use to perform feature selection within this notebook.


### Iris Data

The first data set we use to perform feature selection is the [Iris data][id]. Previously, we used seaborn to load iris data to a dataframe. In this notebook we use scikit learn library which loads the iris data to an object. The object has data and target attributes which contains training features and target label of iris data in numpy array format. These data contain four features: sepal length, sepal width, petal length and petal width, for three different Iris varieties. In total, there are fifty examples of each type of Iris, for 150 total instances in the data set. **To increase the challenge, we will occasionally add random _noise_ features to these data in order to test if a feature selection technique can distinguish between signal and noise.**

-----

[id]: http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [2]:
import sklearn.datasets as ds

# Load Iris Data
iris = ds.load_iris()

# Extract features & labels
features = iris.data
labels = iris.target

print(f'Feature names:{iris.feature_names}')
# Output examples of each class
print(f'Feature {features[0]}: Label {labels[0]}')
print(f'Feature {features[50]}: Label {labels[50]}')
print(f'Feature {features[100]}: Label {labels[100]}')

Feature names:['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Feature [5.1 3.5 1.4 0.2]: Label 0
Feature [7.  3.2 4.7 1.4]: Label 1
Feature [6.3 3.3 6.  2.5]: Label 2


### Adult Income Data
The second data set we use throughout this notebook is the [Adult income prediction task][uciad]. These data were extracted by Barry Becker from the 1994 Census database and consist of the following features: age, workclass, fnlwgt, education, education-level, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, and salary. Of these, only five are continuous:  fnlwgt, education-num, capital-gain, capital-loss, and hours-per-week, the others are discrete. The last column, salary, is discrete and contains one of two strings to indicate if the salary was below or above $50,000. This is the column we will use to make our label.

The following code cell prepares the data:

1. Load data from uci data repository
2. Create label from Salary column
3. Encode categorical features with string value
4. Combine numerical features and encoded categorical features.

-----
[uciad]: https://archive.ics.uci.edu/ml/datasets/Adult

In [3]:
from sklearn.preprocessing import LabelEncoder

data_file = "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
col_names = ['Age', 'Workclass', 'FNLWGT', 'Education', 
             'EducationLevel', 'MaritalStatus', 'Occupation', 
             'Relationship', 'Race', 'Sex', 'CapitalGain', 'CapitalLoss', 
             'HoursPerWeek', 'NativeCountry', 'Salary']

# Read CSV data from URL return Pandas
adult_data = pd.read_csv(data_file, index_col=False, names = col_names)

# Create label column, one for >50K, zero otherwise.
adult_data['Label'] = adult_data['Salary'].map(lambda x : 1 if '>50K' in x else 0)

# Generate categorical features
categorical_features = adult_data[['Workclass', 'Education', 'MaritalStatus', 
               'Occupation', 'Relationship', 'Race', 'Sex', 'NativeCountry']]

categorical_features = categorical_features.apply(LabelEncoder().fit_transform)

# Extract numerical features
numerical_features = adult_data[['Age', 'FNLWGT', 'EducationLevel', 'CapitalGain', 'CapitalLoss', 'HoursPerWeek']]

adult_features = pd.concat([numerical_features, categorical_features], axis=1)

adult_label = adult_data['Label']
adult_features.head()

,Age,FNLWGT,EducationLevel,CapitalGain,CapitalLoss,HoursPerWeek,Workclass,Education,MaritalStatus,Occupation,Relationship,Race,Sex,NativeCountry
0,39,77516,13,2174,0,40,7,9,4,1,1,4,1,39
1,50,83311,13,0,0,13,6,9,2,4,0,4,1,39
2,38,215646,9,0,0,40,4,11,0,6,1,4,1,39
3,53,234721,7,0,0,40,4,1,2,6,0,2,1,39
4,28,338409,13,0,0,40,4,9,2,10,5,2,0,5


-----
[[Back to TOC]](#Table-of-Contents)

## Filter Methods
Filter methods typically involve the application of a statistical measure to score the different features. This score allows the features to be ranked, and this ranking is used to determine which features to keep and which can be removed from the data. Generally each feature is considered on its own (i.e., a univariate test).

### Statistical Tests

One of the simplest techniques for algorithmically selecting features is to measure the variance, or spread, in each feature. Some machine learning algorithms, such as the decision tree, explicitly measure the variance of features and split those features with the greatest variance. The reason for this approach is that features with the greatest variance contain significant information, whereas features with the least variance are tightly bunched and contain little discriminative power. As an extreme example, a feature that has zero variance provides no descriptive power (since all features have the same value) and can easily be removed from analysis without impacting the predictive performance of an algorithm.

Formally, this technique is known as [variance thresholding][vt], which is implemented in the scikit learn library by the [`VarianceThreshold`][skvt] selector. The following two Code cells demonstrate this technique on the Iris data. First, the technique is applied directly to the Iris data, which provides a ranking of feature importance (via the variance measures). 

However, since the original features are unnormalized, the variance comparison is inaccurate, some features have a naturally larger spread due to the sizes of the widths and lengths of the petals and sepals. Thus, the second Code cell normalizes these features to the same zero to one range, and then performance variance thresholding. Notice how the results change such that the petal width becomes more important than the petal length. This example emphasizes the importance of ensuring that the statistical tests are performed in a uniform manner in order to avoid biasing the results. 


-----

[vt]: http://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance
[skvt]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html

In [4]:
# Perform variance thresholding on raw features
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold()

# Compute and display variances
vt.fit_transform(features)
for var, name in zip(vt.variances_, iris.feature_names):
    print(f'{name:>10} variance = {var:5.3f}')

sepal length (cm) variance = 0.681
sepal width (cm) variance = 0.189
petal length (cm) variance = 3.096
petal width (cm) variance = 0.577


In [5]:
# Scale features and then perform variance thresholding
from sklearn.preprocessing import MinMaxScaler
features_ss = MinMaxScaler().fit_transform(features)

# Compute and display variances
vt.fit_transform(features_ss)
for var, name in zip(vt.variances_, iris.feature_names):
    print(f'{name:>10} variance = {var:5.3f}')

sepal length (cm) variance = 0.053
sepal width (cm) variance = 0.033
petal length (cm) variance = 0.089
petal width (cm) variance = 0.100


----

As an additional example, we can apply variance thresholding to the adult income data set. In this case, we display feature variances in descending order.

The feature variances shows that `HoursPerWeek` is among the features with lowest variance. However, we know that `HoursPerWeek` is definitely a very important factor of income. On the other hand, binary categorical features like `Sex` normally have variance close to 0.25 when classes are balanced(50% male and 50% female). This tells us that variance threshold may not be a reliable indicator of feature importance when using alone.

----

In [6]:
# Transform data to [0, 1]
adult_features_ss = MinMaxScaler().fit_transform(adult_features)

# Compute and display variances
vt.fit_transform(adult_features_ss)

for name, var in sorted(zip(adult_features.columns, vt.variances_), key=lambda x: x[1], reverse=True):
    print(f'{name:>18} variance = {var:5.3f}')

               Sex variance = 0.221
      Relationship variance = 0.103
        Occupation variance = 0.091
         Education variance = 0.067
     MaritalStatus variance = 0.063
              Race variance = 0.045
     NativeCountry variance = 0.036
               Age variance = 0.035
         Workclass variance = 0.033
    EducationLevel variance = 0.029
      HoursPerWeek variance = 0.016
       CapitalLoss variance = 0.009
       CapitalGain variance = 0.005
            FNLWGT variance = 0.005


-----
[[Back to TOC]](#Table-of-Contents)

### Univariate Techniques

Another technique for identifying the features that encode the majority of the signal in a data set is to employ a function that computes a statistical measure of the amount of information contained in a given feature. For example, the function might measure the correlation between two features, since correlated features provide redundant information. 

The two main techniques for performing this type of feature selection are [`SelectKBest`][skb] and [`SelectPercentile`][sp]. The former selects the **k** best features, while the latter selects the best percentage of features. Each of these techniques accepts a `score_func` that implements the statistical measure. Provided measures include the following:
- `f_classif`: default value, computes the ANOVA F-value between the features and labels, used for classification.
- `mutual_info_classif`: computes the mutual information of discrete label, used for classification.
- `chi2`: computes chi-squared statistic of non-negative features, used for classification.
- `f_regression`: computes the ANOVA F-value between the features and labels, used for regression.
- `mutual_info_regression`: computes the mutual information for continuous label, used for regression.

Several other specific techniques are also provided by the scikit learn library, but they are beyond the scope of this notebook. The [online documentation][skut] provides more details on all of these methods.

To demonstrate these techniques, we will start with the original Iris data set. We use the `SelectKBest` technique to compute the scores for all features, and we use the default scoring function which is [`f_classif`][fc]. This statistic measures the degree of linear dependence between two features. The more dependent two features are the less information the second feature provides to the classification method. We indicate all features should be kept by setting `k='all'` so that we can print out scores of all features. If we set `k` to a number _n_, then only the best *n* features are kept.

The results indicate that the petal features are most important, which agrees with the feature importance results we saw in earlier notebooks.

-----
[skut]: http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection
[skb]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
[sp]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html
[fc]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html

In [7]:
from sklearn.feature_selection import SelectKBest

skb = SelectKBest(k='all')

fs = skb.fit(features, labels)
for var, name in zip(fs.scores_, iris.feature_names):
    print(f'{name:>18} score = {var:5.3f}')


 sepal length (cm) score = 119.265
  sepal width (cm) score = 49.160
 petal length (cm) score = 1180.161
  petal width (cm) score = 960.007


-----

To test these techniques more thoroughly, we can add random noise features into the analysis. To do this, the following Code cell generates ten new features (called _NoiseXX_ where the XX is replaced by the ordinal number of the new feature) that contain values that are uniformly sampled from the range zero to one. We combine these new noise features with our Iris data, which have been properly normalized to the same range, by using the NumPy `hstack` method, and we also create a new list of feature names that aligns with our new set of features.

Next, we again perform feature selection by using the `SelectKBest` technique. Now, however, we display the features, sorted by their relative importance. In this case, the real features are identified with higher importance.

-----

In [8]:
# Number of noise features to add
num_nf = 10

# Set random state
rng = np.random.RandomState(23)

# Create noise features
noise = rng.uniform(0., 1.0, size=(len(iris.data), num_nf))

# Features plus noise
features_pn = np.hstack((features_ss, noise))

# Feature names
f_names = iris.feature_names.copy()
for i in range(noise.shape[1]):
    f_names.append(f'Noise {i:0>2}')
    
# Fit features plus noise
fs = skb.fit(features_pn, labels)

# Display scores for features and noise
for var, name in sorted(zip(fs.scores_, f_names), key=lambda x: x[0], reverse=True):
    print(f'{name:>18} score = {var:5.3f}')

 petal length (cm) score = 1180.161
  petal width (cm) score = 960.007
 sepal length (cm) score = 119.265
  sepal width (cm) score = 49.160
          Noise 03 score = 7.669
          Noise 09 score = 3.705
          Noise 00 score = 1.888
          Noise 04 score = 1.447
          Noise 08 score = 0.944
          Noise 02 score = 0.554
          Noise 05 score = 0.417
          Noise 06 score = 0.209
          Noise 01 score = 0.105
          Noise 07 score = 0.020


-----

To provide additional insight into these techniques, we now switch to the adult income data set. In the following example, we change the score_func to mutual_info_classif which quantifies the dependency between features and the label. This measure uses a non-parametric (or model free) entropy measurement from the nearest neighbor distances. When two features are independent, this statistic goes to zero, and as the dependency increases the statistic also increases.

Notice that we get a completely different order of importance when comparing to the result from variance threshold. In this case, since the adult data is an imbalanced dataset(25% high income and 75% low income), SelectKBest is a better choice to VarianceThreshold.

-----


In [9]:
from sklearn.feature_selection import mutual_info_classif

skb = SelectKBest(mutual_info_classif, k='all')
fs = skb.fit(adult_features, adult_label)

# Display scores for features and noise
for var, name in sorted(zip(fs.scores_, adult_features.columns), key=lambda x: x[0], reverse=True):
    print(f'{name:>18} score = {var:5.3f}')

      Relationship score = 0.114
     MaritalStatus score = 0.113
       CapitalGain score = 0.083
         Education score = 0.067
    EducationLevel score = 0.066
               Age score = 0.065
        Occupation score = 0.060
      HoursPerWeek score = 0.041
            FNLWGT score = 0.034
       CapitalLoss score = 0.034
               Sex score = 0.030
         Workclass score = 0.021
     NativeCountry score = 0.011
              Race score = 0.009


-----
[[Back to TOC]](#Table-of-Contents)

## Wrapper methods
Wrapper methods consider the selection of a set of features as a search problem, where different combinations are prepared, evaluated and compared to other combinations. A predictive model is used to evaluate a combination of features and assign a score based on model accuracy. One popular wrapper method is the recursive feature elimination algorithm.


### Recursive Feature Extraction

[Recursive Feature Elimination (RFE)][rfe] works by recursively removing attributes and building a model from the remaining attributes. The model accuracy is used to identify the attributes (and combination of attributes) that most contribute to predicting the target (or held-out) attribute. The RFE implementation provided by the scikit learn library is in the `feature_selection` module.

In the next few cells, we employ both RFE to determine the most important features for both the Iris and adult income  data. The first Code cell below uses a linear support vector classifier to perform RFE. In this case, we analyze the Iris data set plus ten _noise_ features. The result of this operation identifies the most important feature (since we specified that the top feature should be identified),  but also ranks the remaining features. Notice how three of the _real_ features are the top three ranked features, but next are several _noise_ features, indicating the remaining _real_ feature encodes less information than a random feature.

-----

[rfe]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html



In [10]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE

# Create classifier
svc = LinearSVC(random_state=23)

# Perform RFE, select top feature (but rank all)
rfe = RFE(svc, 1)

# Fit features plus noise
fs = rfe.fit(features_pn, labels)
    
# Display scores for features and noise
for var, name in sorted(zip(fs.ranking_, f_names), key=lambda x: x[0]):
    print(f'{name:>18} rank = {var:5.3f}')

  petal width (cm) rank = 1.000
 petal length (cm) rank = 2.000
  sepal width (cm) rank = 3.000
          Noise 03 rank = 4.000
          Noise 09 rank = 5.000
          Noise 00 rank = 6.000
 sepal length (cm) rank = 7.000
          Noise 07 rank = 8.000
          Noise 05 rank = 9.000
          Noise 06 rank = 10.000
          Noise 02 rank = 11.000
          Noise 08 rank = 12.000
          Noise 04 rank = 13.000
          Noise 01 rank = 14.000


-----

We now transition to the adult income data set. The following code cell takes considerable time to run. The reason is that RFE trains the selected model(RandomeForestClassifier in this case) repeatly to rank the features. It'll take a while when the dataset is large.

-----

In [11]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=23)
# Create RFE model with only one feature
rfe = RFE(rfc, 1)
fs = rfe.fit(adult_features, adult_label)

# Display scores for features and noise
for var, name in sorted(zip(fs.ranking_, adult_features.columns), key=lambda x: x[0]):
    print(f'{name:>18} rank = {var:5.3f}')

            FNLWGT rank = 1.000
      Relationship rank = 2.000
       CapitalGain rank = 3.000
               Age rank = 4.000
    EducationLevel rank = 5.000
      HoursPerWeek rank = 6.000
        Occupation rank = 7.000
         Workclass rank = 8.000
     MaritalStatus rank = 9.000
       CapitalLoss rank = 10.000
         Education rank = 11.000
     NativeCountry rank = 12.000
               Sex rank = 13.000
              Race rank = 14.000


-----

<font color='red' size = '5'> Student Exercise </font>

Now that you have run the previous cells, try making changes to the notebook:

1. Try using a different classifier, such as a decision tree or logistic regression.

-----

[[Back to TOC]](#Table-of-Contents)

## Embedded Methods
Embedded Methods perform feature selection directly in the model construction. For example, decision tree-based techniques compute feature importance. This extra information can be used to rank features for use with these models.

## Select From Model

Of course some algorithms, such as the decision tree and the ensemble techniques based on a decision tree, provide access to measures of the feature importance. For example, the [Random Forest Classifier (RFC)][rfc], as an ensemble method, builds models by randomly selecting features when building each tree. In this process, RFC computes the overall importance of each feature in building the final model. By extracting the feature importances from the final model, we obtain a ranked ordering of the features used to build the model.

In the following Code cell, we employ RFC to determine the most important features for the Iris data set (along with the added _noise_ features). With this estimator, the four _real_ features are all ranked as the most important, followed by the _noise_ features. However, the sepal features are ranked with low importance, nearly the same as the noise features. This explains why different techniques can produce different feature rankings, and small changes to the algorithm or any inherent randomness in the process can generate different results.

-----

[rfc]: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

-----

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Build model
rfc = RandomForestClassifier(random_state=23)
rfc.fit(features_pn, labels)

# Display scores for features and noise
print(f'{"Label":18s}: Importance')
print(26*'-')
for val, name in sorted(zip(rfc.feature_importances_, f_names), 
                        key=lambda x: x[0], reverse=True):
    print(f'{name:>18}: {100.0*val:05.2f}%')

Label             : Importance
--------------------------
 petal length (cm): 40.89%
  petal width (cm): 24.89%
 sepal length (cm): 06.57%
  sepal width (cm): 04.34%
          Noise 04: 04.11%
          Noise 03: 03.62%
          Noise 02: 03.38%
          Noise 00: 02.80%
          Noise 01: 02.29%
          Noise 05: 02.22%
          Noise 09: 01.99%
          Noise 07: 01.47%
          Noise 06: 01.19%
          Noise 08: 00.25%


-----

We now use randome forest classifier to determine feature importance in  the adult income data. 

-----
[sfm]: http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html

In [13]:
rfc = RandomForestClassifier(random_state=23)
rfc.fit(adult_features, adult_label)

# Display scores for features and noise
print(f'{"Label":18s}: Importance')
print(26*'-')
for val, name in sorted(zip(rfc.feature_importances_, adult_features.columns), 
                        key=lambda x: x[0], reverse=True):
    print(f'{name:>18}: {100.0*val:05.2f}%')

Label             : Importance
--------------------------
            FNLWGT: 17.25%
               Age: 14.53%
       CapitalGain: 11.39%
    EducationLevel: 09.94%
      Relationship: 09.46%
      HoursPerWeek: 08.02%
     MaritalStatus: 07.29%
        Occupation: 06.72%
       CapitalLoss: 03.91%
         Workclass: 03.91%
         Education: 03.00%
     NativeCountry: 01.85%
               Sex: 01.39%
              Race: 01.34%


-----

<font color='red' size = '5'> Student Exercise </font>

Now that you have run the previous cells, try making changes to the
notebook:

1. Try changing some hyperparameters(ie. max_depth) for the RFC estimator. How does these changes affect the feature importance?

-----

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Detailed discussion of [feature engineering][1]
1. Series of blog articles on feature selection in Python: [Part I][2a], [Part II][2b], [Part II][2c], and [Part IV][2d]
2. An introduction to [feature selection][3]


-----

[1]: http://adataanalyst.com/machine-learning/comprehensive-guide-feature-engineering/
[2a]: http://blog.datadive.net/selecting-good-features-part-i-univariate-selection/
[2b]: http://blog.datadive.net/selecting-good-features-part-ii-linear-models-and-regularization/
[2c]: http://blog.datadive.net/selecting-good-features-part-iii-random-forests/
[2d]: http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/

[3]: https://machinelearningmastery.com/an-introduction-to-feature-selection/

**&copy; 2019: Gies College of Business at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode